In [ ]:
 import cv2.aruco as aruco

In [ ]:
 aruco_dict = aruco.Dictionary_get(aruco.DICT_4X4_50)

In [ ]:
import cv2
import numpy as np
import time
import cv2.aruco as aruco

aruco_dict = aruco.Dictionary_get(aruco.DICT_4X4_50)

# Define the parameters for the marker detection
parameters = aruco.DetectorParameters_create()
M = None

# A required callback method that goes into the trackbar function.
def nothing(x):
    pass


def get_rectangle_coordinates(aruco_marker_corners1, aruco_marker_corners2):
    # Define the scale factor for the bounding box
    scale_factor = 5
    aruco1_coords = aruco_marker_corners1
    aruco2_coords = aruco_marker_corners2

    # Combine the coordinates of the two markers into a single array
    rectangle_coords = np.array([aruco1_coords[3], aruco2_coords[1]], dtype=np.int32)

    return rectangle_coords
    # return np.array([corner1, corner2, corner3, corner4], dtype=np.int32)


# Initializing the webcam feed.
#cap = cv2.VideoCapture("http://192.168.1.90")
cap = cv2.VideoCapture("http://192.168.1.90:8080/video")
cap.set(3, 640)
cap.set(4, 480)
wname = "Trackbars"
# Create a window named trackbars.
cv2.namedWindow(wname)

wname1 = "image"
cv2.namedWindow(wname1)

l_h = 0
l_s = 0
l_v = 0
u_h = 179
u_s = 255
u_v = 255
hsv = None


def pixel_mm(u,v):
    sccale_factor = (12*25.4)/500
    x_cord = u*sccale_factor
    y_coord = v *sccale_factor
    return x_cord,y_coord


def onclick(event, x, y, flags, param):
    global u_s, u_v, u_h, l_h, l_s, l_v, hsv
    if event == cv2.EVENT_LBUTTONDOWN:
        print("x: %d, y: %d" % (x, y))

        print(hsv[y, x][0])
        l_h = max(hsv[y, x][0] - 10, 0)
        l_s = max(hsv[y, x][1] - 40, 0)
        l_v = max(hsv[y, x][2] - 40, 0)
        u_h = min(hsv[y, x][0] + 10, 179)
        u_s = min(hsv[y, x][1] + 40, 255)
        u_v = min(hsv[y, x][2] + 40, 255)
        # cv2.setTrackbarPos("L - H", wname, l_h)
        # cv2.setTrackbarPos("L - S", wname, l_s)
        # cv2.setTrackbarPos("L - V", wname, l_v)
        # cv2.setTrackbarPos("U - H", wname, u_h)
        # cv2.setTrackbarPos("U - S", wname, u_s)
        # cv2.setTrackbarPos("U - V", wname, u_v)
        print("values updated ")
        # x_cord,y_cord = pixel_mm(x,y)
        # print(f"x cord is {x_cord} and y cord is {y_cord}")


rectangle = None
# cv2.setMouseCallback(wname, onclick)
cv2.setMouseCallback(wname1, onclick)
# Now create 6 trackbars that will control the lower and upper range of
# H,S and V channels. The Arguments are like this: Name of trackbar,
# window name, range,callback function. For Hue the range is 0-179 and
# for S,V its 0-255.
# cv2.createTrackbar("L - H", "Trackbars", 0, 179, nothing)
# cv2.createTrackbar("L - S", "Trackbars", 0, 255, nothing)
# cv2.createTrackbar("L - V", "Trackbars", 0, 255, nothing)
# cv2.createTrackbar("U - H", "Trackbars", 179, 179, nothing)
# cv2.createTrackbar("U - S", "Trackbars", 255, 255, nothing)
# cv2.createTrackbar("U - V", "Trackbars", 255, 255, nothing)

while True:

    # Start reading the webcam feed frame by frame.
    ret, frame = cap.read()
    # frame = cv2.imread('/Users/anujvadecha/Desktop/frame.png')

    if not ret:
        break

    frame = cv2.resize(frame, (480 * 2, 270 * 2))

    corners, ids, rejected = aruco.detectMarkers(frame, aruco_dict, parameters=parameters)


    output_coords = np.float32([[0, 0], [500, 0], [500, 300], [0, 300]])
    if len(corners) == 4 and M is None:
        # Define the source and destination points for perspective transformation
        src_pts = np.float32([corners[np.where(ids == 3)[0][0]][0][0], corners[np.where(ids == 2)[0][0]][0][1],
                              corners[np.where(ids == 0)[0][0]][0][2], corners[np.where(ids == 1)[0][0]][0][3]])
        dst_pts = np.float32([[0, 0], [0, 500], [500, 500], [500, 0]])

        # Get the perspective transform matrix
        M = cv2.getPerspectiveTransform(src_pts, dst_pts)
    if M is not None:

        # Apply perspective transformation to the input image
        warped = cv2.warpPerspective(frame, M, (500, 500))
        frame =warped



    # Draw the detected markers on the frame
    aruco.drawDetectedMarkers(frame, corners, ids)

    # Flip the frame horizontally (Not required)
    # frame = cv2.flip(frame, 1)
    # Convert the BGR image to HSV image.
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower_range = np.array([l_h, l_s, l_v])
    upper_range = np.array([u_h, u_s, u_v])
    # Filter the image and get the binary mask, where white represents
    # your target color
    mask = cv2.inRange(hsv, lower_range, upper_range)
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    contoursa, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    ball_count = 0
    #print("total:",ball_count)

    #print("a:",len(contoursa))
    # Iterate through the contours
    for cnt in contours:
        # Check if the contour is a rectangle
        approx = cv2.approxPolyDP(cnt, 0.04 * cv2.arcLength(cnt, True), True)
        if len(approx) >= 3 and cv2.isContourConvex(approx):
            # Draw a bounding box around the rectangle
            rect = cv2.minAreaRect(cnt)
            box = cv2.boxPoints(rect)
            area = cv2.contourArea(cnt)
            perimeter = cv2.arcLength(cnt, True)
            circularity = 4 * np.pi * area / (perimeter ** 2)
            print(area, perimeter)
            if area > 1000 and circularity > 200:
                ball_count += 1
            box = np.int0(box)
            # Compute the angle of rotation of the rectangle
            angle = rect[2]
            cv2.drawContours(frame, [box], 0, (0, 255, 0), 2)
            moments = cv2.moments(cnt)

            # Calculate the centroid coordinates
            centroid_x = int(moments['m10'] / moments['m00'])
            centroid_y = int(moments['m01'] / moments['m00'])
            cv2.circle(frame, (centroid_x, centroid_y), 5, (0, 255, 0), -1)
            #print(centroid_y,centroid_y)
    print("total:",ball_count)
    print(("a:",len(contours)))
            # x, y, w, h = cv2.boundingRect(approx)
            # cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    # You can also visualize the real part of the target color (Optional)
    # res = cv2.bitwise_and(frame, frame, mask=mask)
    # Converting the binary mask to 3 channel image, this is just so
    # we can stack it with the others
    # mask_3 = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    # stack the mask, orginal frame and the filtered result
    # stacked = np.hstack((mask_3, frame, res))
    # Show this stacked frame at 40% of the size.
    cv2.imshow('Trackbars', mask)
    frame = cv2.putText(frame, "X", (382, 307), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 3)
    cv2.imshow(wname1, frame)
    # If the user presses ESC then exit the program
    key = cv2.waitKey(1)
    if key == 27:
        break
    # If the user presses `s` then print this array.
    if key == ord('s'):
        thearray = [[l_h, l_s, l_v], [u_h, u_s, u_v]]
        print(thearray)
        # Also save this array as penval.npy
        np.save('hsv_value', thearray)
        break
    if key == ord('r'):
        rectange = box

# Release the camera & destroy the windows.
cap.release()
cv2.destroyAllWindows()

In [ ]:
from uarm.wrapper import SwiftAPI
import time

In [ ]:
accessed = False
while not accessed:
    try:
        swift = SwiftAPI()
        accessed = True
    except Exception as e:
        time.sleep(0.2)

In [ ]:
swift = SwiftAPI()

In [ ]:
swift.connect()

In [ ]:
swift.get_device_info()

In [1]:
import cv2
import numpy as np
import time
import cv2.aruco as aruco
from uarm.wrapper import SwiftAPI
import time
import queue, threading, time
swift = SwiftAPI()
#swift.connect()
aruco_dict = aruco.Dictionary_get(aruco.DICT_4X4_50)
parameters = aruco.DetectorParameters_create()
M = None

# bufferless VideoCapture
class VideoCapture:
    def __init__(self, name):
        self.cap = cv2.VideoCapture(name)
        self.q = queue.Queue()
        t = threading.Thread(target=self._reader)
        t.daemon = True
        t.start()

  # read frames as soon as they are available, keeping only most recent one
    def _reader(self):
        while True:
            ret, frame = self.cap.read()
            if not ret:
                break
            if not self.q.empty():
                try:
                    self.q.get_nowait()   # discard previous (unprocessed) frame
                except queue.Empty:
                    pass
                self.q.put(frame)
            key = cv2.waitKey(1)
            if key == 27:
                break
          

    def read(self):
        return self.q.get()

def nothing(x):
    pass
def move_robot(u,v):
    x = round(u,2)
    y = round(v,2)
    swift.set_position(x=x, y=y, z=100, speed=100000)
    time.sleep(1.0)
    swift.set_position(x=x, y=y,z =40 ,speed=10000)
    time.sleep(1.0)
    swift.set_pump(on=True)
    time.sleep(2)
    swift.set_position(x=x, y=x, z=100, speed=100000)
    time.sleep(1)
    swift.set_position(x=0, y=-100, z=100, speed=100000)
    time.sleep(1.0)
    swift.set_position(x=0, y=-100, z=70, speed=100000)
    time.sleep(1.0)
    swift.set_pump(on=False)
    time.sleep(1.0)
    swift.set_position(x=0, y=0, z=100, speed=100000)


def get_rectangle_coordinates(aruco_marker_corners1, aruco_marker_corners2):
    scale_factor = 5
    aruco1_coords = aruco_marker_corners1
    aruco2_coords = aruco_marker_corners2
    rectangle_coords = np.array([aruco1_coords[3], aruco2_coords[1]], dtype=np.int32)
    return rectangle_coords
cap = cv2.VideoCapture("http://192.168.1.90:8080/video")
#cap = VideoCapture("http://10.84.18.188:8080/video")
wname = "Trackbars"
cv2.namedWindow(wname)
wname1 = "image"
cv2.namedWindow(wname1)

l_h = 0
l_s = 0
l_v = 0
u_h = 179
u_s = 255
u_v = 255
hsv = None


def pixel_mm(u,v):
    sccale_factor = (12*25.4)/500
    x_cord = u*sccale_factor
    y_coord = v *sccale_factor
    return x_cord,y_coord


def onclick(event, x, y, flags, param):
    global u_s, u_v, u_h, l_h, l_s, l_v, hsv
    if event == cv2.EVENT_LBUTTONDOWN:
        print("x: %d, y: %d" % (x, y))
        #print(hsv[y, x][0])
        l_h = max(hsv[y, x][0] - 10, 0)
        l_s = max(hsv[y, x][1] - 40, 0)
        l_v = max(hsv[y, x][2] - 40, 0)
        u_h = min(hsv[y, x][0] + 10, 179)
        u_s = min(hsv[y, x][1] + 40, 255)
        u_v = min(hsv[y, x][2] + 40, 255)
        print("values updated ")
rectangle = None
cv2.setMouseCallback(wname1, onclick)
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.resize(frame, (480 * 2, 270 * 2))
    corners, ids, rejected = aruco.detectMarkers(frame, aruco_dict, parameters=parameters)
    output_coords = np.float32([[0, 0], [500, 0], [500, 300], [0, 300]])
    if len(corners) == 4 and M is None:
        src_pts = np.float32([corners[np.where(ids == 3)[0][0]][0][0], corners[np.where(ids == 2)[0][0]][0][1],
                              corners[np.where(ids == 0)[0][0]][0][2], corners[np.where(ids == 1)[0][0]][0][3]])
        dst_pts = np.float32([[0, 0], [0, 500], [500, 500], [500, 0]])
        M = cv2.getPerspectiveTransform(src_pts, dst_pts)
    if M is not None:
        warped = cv2.warpPerspective(frame, M, (500, 500))
        frame = warped
    aruco.drawDetectedMarkers(frame, corners, ids)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower_range = np.array([l_h, l_s, l_v])
    upper_range = np.array([u_h, u_s, u_v])
    mask = cv2.inRange(hsv, lower_range, upper_range)
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    ball_count = 0
    cordinates = []
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 200 and area <10000:
            (x, y), radius = cv2.minEnclosingCircle(contour)
            center = (int(x), int(y))
            radius = int(radius)
            moments = cv2.moments(contour)
            cv2.circle(frame, center, radius, (0, 255, 0), 2)
            centroid_x = int(moments['m10'] / moments['m00'])
            centroid_y = int(moments['m01'] / moments['m00'])
            cv2.circle(frame, (centroid_x, centroid_y), 5, (0, 255, 255), -1)
            ball_count += 1
            cordinates.append(center)
            if cordinates is not None:
                for i in cordinates:
                    u,v = i
                    x,y = pixel_mm(u,v)
                    print("length:",len(cordinates))
                    print("moving to:",(x,y))
                    move_robot(x,y)
                cordinates =[]
   
    cv2.imshow('Trackbars', mask)
    frame = cv2.putText(frame, "X", (250, 250), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 3)
    cv2.imshow(wname1, frame)
    #if len(cordinates)>1:
    print(len(cordinates))
    #print(len(cordinates))
    key = cv2.waitKey(1)
    if key == 27:
        break
    if key == ord('s'):
        thearray = [[l_h, l_s, l_v], [u_h, u_s, u_v]]
        print(thearray)
        np.save('hsv_value', thearray)
        break
    if key == ord('r'):
        rectange = box
cap.release()
cv2.destroyAllWindows()

Exception: can not found port, please connect the port via usb

In [ ]:
import cv2
cap = cv2.VideoCapture("http://10.84.18.188:8080/video")

In [ ]:
#cap.release()
cv2.destroyAllWindows()

In [21]:
from uarm.wrapper import SwiftAPI
swift = SwiftAPI()
#swift.set_position(x=0, y=100, z=100, speed=10000)
swift.reset()

In [24]:
swift.set_position(x=210, y=100, z=100, speed=10000)

In [ ]:
from uarm.wrapper import SwiftAPI
swift = SwiftAPI()
import cv2, queue, threading, time
import cv2
import numpy as np
import time
import cv2.aruco as aruco
from uarm.wrapper import SwiftAPI
import time
import queue, threading, time

aruco_dict = aruco.Dictionary_get(aruco.DICT_4X4_50)
parameters = aruco.DetectorParameters_create()
M = None
wname = "Trackbars"
cv2.namedWindow(wname)
wname1 = "image"
cv2.namedWindow(wname1)

l_h = 0
l_s = 0
l_v = 0
u_h = 179
u_s = 255
u_v = 255
hsv = None




def get_rectangle_coordinates(aruco_marker_corners1, aruco_marker_corners2):
    scale_factor = 5
    aruco1_coords = aruco_marker_corners1
    aruco2_coords = aruco_marker_corners2
    rectangle_coords = np.array([aruco1_coords[3], aruco2_coords[1]], dtype=np.int32)
    return rectangle_coords


def move_robot(u, v):
    x = 5+round(u, 2)
    y = -5+round(v, 2)
    swift.set_position(x=x, y=y, z=100, speed=10000)
    time.sleep(1.0)
    swift.set_position(x=x, y=y, z=40, speed=10000)
    time.sleep(1.0)
    swift.set_pump(on=True)
    time.sleep(2)
    swift.set_position(x=x, y=x, z=100, speed=10000)
    time.sleep(1)
    swift.set_position(x=0, y=-100, z=100, speed=10000)
    time.sleep(1.0)
    swift.set_position(x=0, y=-100, z=70, speed=10000)
    time.sleep(1.0)
    swift.set_pump(on=False)
    time.sleep(1.0)
    swift.set_position(x=0, y=0, z=100, speed=10000)


# bufferless VideoCapture
class VideoCapture:

    def __init__(self, name):
        self.cap = cv2.VideoCapture(name)
        self.q = queue.Queue()
        t = threading.Thread(target=self._reader)
        t.daemon = True
        t.start()

    # read frames as soon as they are available, keeping only most recent one
    def _reader(self):
        while True:
            ret, frame = self.cap.read()
            if not ret:
                break
            if not self.q.empty():
                try:
                    self.q.get_nowait()   # discard previous (unprocessed) frame
                except queue.Empty:
                    pass
            self.q.put(frame)

    def read(self):
        return self.q.get()


def pixel_mm(u,v):
    sccale_factor = (12*25.4)/500
    x_cord = u*sccale_factor
    y_coord = v *sccale_factor
    return x_cord,y_coord


def onclick(event, x, y, flags, param):
    global u_s, u_v, u_h, l_h, l_s, l_v, hsv
    if event == cv2.EVENT_LBUTTONDOWN:
        print("x: %d, y: %d" % (x, y))
        # print(hsv[y, x][0])
        l_h = max(hsv[y, x][0] - 10, 0)
        l_s = max(hsv[y, x][1] - 40, 0)
        l_v = max(hsv[y, x][2] - 40, 0)
        u_h = min(hsv[y, x][0] + 10, 179)
        u_s = min(hsv[y, x][1] + 40, 255)
        u_v = min(hsv[y, x][2] + 40, 255)
    

rectangle = None
cv2.setMouseCallback(wname1, onclick)

#swift.connect()
#cap = VideoCapture("http://10.84.18.188:8080/video")
cap = VideoCapture("http://192.168.1.90:8080/video")
while True:
  # time.sleep(.5)   # simulate time between events
    frame = cap.read()
    frame = cv2.resize(frame, (480 * 2, 270 * 2))
    corners, ids, rejected = aruco.detectMarkers(frame, aruco_dict, parameters=parameters)
    output_coords = np.float32([[0, 0], [500, 0], [500, 300], [0, 300]])
    if len(corners) == 4 and M is None:
        src_pts = np.float32([corners[np.where(ids == 3)[0][0]][0][0], corners[np.where(ids == 2)[0][0]][0][1],
                          corners[np.where(ids == 0)[0][0]][0][2], corners[np.where(ids == 1)[0][0]][0][3]])
        dst_pts = np.float32([[0, 0], [0, 500], [500, 500], [500, 0]])
        M = cv2.getPerspectiveTransform(src_pts, dst_pts)
    if M is not None:
        warped = cv2.warpPerspective(frame, M, (500, 500))
        frame = warped
    aruco.drawDetectedMarkers(frame, corners, ids)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower_range = np.array([l_h, l_s, l_v])
    upper_range = np.array([u_h, u_s, u_v])
    mask = cv2.inRange(hsv, lower_range, upper_range)
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    ball_count = 0
    cordinates = []
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 200 and area < 10000:
            (x, y), radius = cv2.minEnclosingCircle(contour)
            center = (int(x), int(y))
            radius = int(radius)
            moments = cv2.moments(contour)
            cv2.circle(frame, center, radius, (0, 255, 0), 2)
            centroid_x = int(moments['m10'] / moments['m00'])
            centroid_y = int(moments['m01'] / moments['m00'])
            cv2.circle(frame, (centroid_x, centroid_y), 5, (0, 255, 255), -1)
            ball_count += 1
            cordinates.append(center)
            if cordinates is not None:
                for i in cordinates:
                    u, v = i
                    x, y = pixel_mm(u, v)
                    print("length:", len(cordinates))
                    print("moving to:", (x, y))
                    move_robot(x,y)
                cordinates = []

    cv2.imshow('Trackbars', mask)
    frame = cv2.putText(frame, "X", (250, 250), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 3)
    cv2.imshow(wname1, frame)
    if chr(cv2.waitKey(1)&255) == 'q':
        break




In [ ]:
import cv2
import cv2, queue, threading, time
import cv2
import numpy as np
import time
import cv2.aruco as aruco
from uarm.wrapper import SwiftAPI
import time
import queue, threading, time
#cap.release()
#cap = cv2.VideoCapture("http://10.84.18.188:8080/video")
cap = cv2.VideoCapture("http://192.168.1.90:8080/video")
while True:
  # time.sleep(.5)   # simulate time between events
    ret,frame = cap.read()
    cv2.imshow("a",frame)
    print(frame.shape)
    if chr(cv2.waitKey(1)&255) == 'q':
        break
cv2.destroyAllWindows()

In [ ]:
cv2.destroyAllWindows()